In [1]:
from sknn import mlp
from sknn.mlp import Classifier, Convolution, Layer

import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer,LabelBinarizer
from sklearn.cross_validation import train_test_split
%matplotlib inline


Couldn't import dot_parser, loading of dot files will not be possible.


Using gpu device 0: GeForce 920M (CNMeM is disabled, cuDNN not available)
/home/mrafi123/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import theano

In [2]:
X_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_train.npy')
X_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/X_test.npy')
y_train=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_train.npy')
y_test=np.load('/media/mrafi123/UStore/Dimensionality-Reduction/data/CIFAR10/y_test.npy')

In [3]:
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [4]:
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print (x_train.shape)
print (x_test.shape)

(50000, 3072)
(10000, 3072)


In [5]:
n_feat = x_train.shape[1]
n_targets = y_train.max() + 1
print(n_feat)
print(n_targets)

3072
10


In [6]:
errors = []
def store_stats(avg_valid_error, avg_train_error, **_):
    errors.append((avg_valid_error, avg_train_error))
    print ('Average Validation Error ',avg_valid_error)
    print ('Average Training Error ',avg_train_error)


In [7]:
#be careful of the humungous verbose
def my_callback(event, **variables):
    print(event)        # The name of the event, as shown in the list above.
    print(variables)    # Full dictionary of local variables from training loop.


In [52]:
#be careful of the humungous verbose
def my_callback(**variables):
    #print(variables)    # Full dictionary of local variables from training loop.
    print('best_params',len(variables['best_params']))
    print('best_params 0',len(variables['best_params'][0]))
    print('best_params 0 0',len(variables['best_params'][0][0]))
    print('best_params 0 1',len(variables['best_params'][0][1]))
    print('best_params 1',len(variables['best_params'][1]))
    print('best_params 1 0',len(variables['best_params'][1][0]))
    print('best_params 1 1',len(variables['best_params'][1][1]))



In [8]:
def my_callback(**variables):
#def my_callback(event, **variables):
    #print(event)        # The name of the event, as shown in the list above.
    print(variables['y'])    # Full dictionary of local variables from training loop.
    print(variables['yb'])
    expected=variables['yb']
    predicted = variables['y']
    num=len(expected)
    r=0
    w=0
    for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(expected[i],predicted[i]):
            r+=1
        else:
            w+=1
    print ("Trained ",  num, "Cifar 10 images")
    print ("correct: ", r, "wrong: ", w, "Training error%: ", float(w)*100/(r+w), "%")
    print ("Training Accuracy ", float(r)*100/(r+w), "%")

In [9]:
print (x_train[49000:].shape)
print (y_train[49000:].shape)
print (x_test[9000:].shape)
print (y_test[9000:].shape)

sample_train = x_train[49000:]
sample_test = x_test[9000:]
sample_y_train = y_train[49000:]
sample_y_test = y_test[9000:]
print (sample_train.shape)
print (sample_test.shape)
print (sample_y_train.shape)
print (sample_y_test.shape)


(1000, 3072)
(1000,)
(1000, 3072)
(1000,)
(1000, 3072)
(1000, 3072)
(1000,)
(1000,)


In [8]:
n_feat = sample_train.shape[1]
n_targets = sample_y_train.max() + 1
print(n_feat)
print(n_targets)

NameError: name 'sample_train' is not defined

In [53]:
samplenn = mlp.Classifier(
        layers=[
            mlp.Layer("Sigmoid", units=n_feat/16),
            mlp.Layer("Softmax", units=n_targets)],
        n_iter=1,
        n_stable=10,
        batch_size=250,
        learning_rate=0.002,
        learning_rule="momentum",
        valid_size=0.1,
        verbose=1,
        callback={'on_epoch_finish': my_callback}
)

In [54]:
nn.fit(sample_train, sample_y_train)


/home/mrafi123/anaconda3/lib/python3.5/site-packages/lasagne/init.py:99: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  low=self.range[0], high=self.range[1], size=shape))
/home/mrafi123/anaconda3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


best_params 2
best_params 0 2
best_params 0 0 3072
best_params 0 1 192
best_params 1 2
best_params 1 0 192
best_params 1 1 10


Classifier(batch_size=250,
      callback={'on_epoch_finish': <function my_callback at 0x7f46a614e9d8>},
      debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Layer `Sigmoid`: frozen=False, units=192.0, name='hidden0'>,
      layers=[<sknn.nn.Layer `Sigmoid`: frozen=False, units=192.0, name='hidden0'>, <sknn.nn.Layer `Softmax`: frozen=False, units=10, name='output'>],
      learning_momentum=0.9, learning_rate=0.002, learning_rule='momentum',
      loss_type=None, n_iter=1, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Softmax`: frozen=False, units=10, name='output'>,
      parameters=None, random_state=None, regularize=None,
      valid_set=(array([[ 0.61961,  0.55294, ...,  1.     ,  1.     ],
       [ 0.71373,  0.76863, ...,  0.61569,  0.58039],
       ...,
       [ 0.92941,  0.92941, ...,  0.64706,  0.71373],
       [ 0.18824,  0.3098 , ...,  0.23137,  0.27451]], dtype=float32), array([[ 0.,  0., ...,  0.,  0.],
       [ 0.,  0., ...,  1.,  0.],
 

In [12]:
dnn = mlp.Classifier(
        layers=[
            mlp.Layer("Rectifier", units=1500),
            mlp.Layer("Rectifier", units=750),
            mlp.Layer("Rectifier", units=350),
            mlp.Layer("Rectifier", units=150),
            mlp.Layer("Softmax", units=n_targets)],
        n_iter=25,
        n_stable=10,
        batch_size=25,
        learning_rate=0.002,
        learning_rule="momentum",
        valid_size=0.1,
        verbose=1,
        callback={'on_epoch_finish': store_stats}
)

In [7]:
print (x_train.shape)
print (x_test.shape)

(50000, 3072)
(10000, 3072)


In [8]:
x_train = x_train.reshape((len(x_train),32,32,3))
x_test = x_test.reshape((len(x_test),32,32,3))

print (x_train.shape)
print (x_test.shape)                     

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [9]:
from sknn.mlp import Classifier, Convolution, Layer
# Deep CNN Architecture from Karpathy JS demo in CIFAR10
dcnn = Classifier(
    layers=[
        Convolution("Rectifier", channels=16, kernel_shape=(5,5),border_mode='full',pool_shape=(2,2)),
        Convolution("Rectifier", channels=20, kernel_shape=(5,5),border_mode='full',pool_shape=(2,2)),
        Convolution("Rectifier", channels=20, kernel_shape=(5,5),border_mode='full',pool_shape=(2,2)),
        Layer('Rectifier', units=64),
        Layer("Softmax",units=n_targets)],
        n_iter=160,
        n_stable=10,
        batch_size=25,
        learning_rate=0.002,
        learning_rule="momentum",
        valid_size=0.1,
        verbose=1,
        callback={'on_epoch_finish': store_stats}
    )

In [10]:
dcnn.fit(x_train, y_train)


/home/mrafi123/anaconda3/lib/python3.5/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/home/mrafi123/anaconda3/lib/python3.5/site-packages/lasagne/layers/conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


Average Validation Error  2.13139769197
Average Training Error  2.21702644361
Average Validation Error  2.00949050725
Average Training Error  2.06345539424
Average Validation Error  1.93037887871
Average Training Error  1.96265527275
Average Validation Error  1.86741361022
Average Training Error  1.88665998532
Average Validation Error  1.8192131865
Average Training Error  1.83021255354
Average Validation Error  1.77397949398
Average Training Error  1.7859295706
Average Validation Error  1.74087003648
Average Training Error  1.7501551089
Average Validation Error  1.72885021687
Average Training Error  1.71876978808
Average Validation Error  1.68984403491
Average Training Error  1.69105318513
Average Validation Error  1.66911869884
Average Training Error  1.66597346683
Average Validation Error  1.64969876289
Average Training Error  1.64418645084
Average Validation Error  1.62939418674
Average Training Error  1.6242887195
Average Validation Error  1.61686390102
Average Training Error  1.60

Classifier(batch_size=25,
      callback={'on_epoch_finish': <function store_stats at 0x7f69b0035d90>},
      debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden0', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=16, border_mode='full', kernel_shape=(5, 5)>,
      hidden1=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden1', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden2=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden2', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden3=<sknn.nn.Layer `Rectifier`: frozen=False, name='hidden3', units=64>,
      layers=[<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden0', kernel_stride=(1, 1), pool_s

In [11]:
dcnn

Classifier(batch_size=25,
      callback={'on_epoch_finish': <function store_stats at 0x7f69b0035d90>},
      debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden0', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=16, border_mode='full', kernel_shape=(5, 5)>,
      hidden1=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden1', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden2=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden2', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden3=<sknn.nn.Layer `Rectifier`: frozen=False, name='hidden3', units=64>,
      layers=[<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden0', kernel_stride=(1, 1), pool_s

In [12]:


from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = dcnn.predict(x_test)

print("Classification report for classifier %s:\n%s\n" % (
    dcnn, classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % confusion_matrix(expected, predicted))

[(10000, 10)]
Classification report for classifier Classifier(batch_size=25,
      callback={'on_epoch_finish': <function store_stats at 0x7f69b0035d90>},
      debug=False, dropout_rate=None, f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden0', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=16, border_mode='full', kernel_shape=(5, 5)>,
      hidden1=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden1', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden2=<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type='max', name='hidden2', kernel_stride=(1, 1), pool_shape=(2, 2), scale_factor=(1, 1), channels=20, border_mode='full', kernel_shape=(5, 5)>,
      hidden3=<sknn.nn.Layer `Rectifier`: frozen=False, name='hidden3', units=64>,
      layers=[<sknn.nn.Convolution `Rectifier`: frozen=False, pool_type=

In [13]:
errors

[(2.1313976919651032, 2.2170264436139),
 (2.0094905072450637, 2.063455394241545),
 (1.9303788787126541, 1.9626552727487352),
 (1.8674136102199554, 1.8866599853171242),
 (1.8192131865024566, 1.8302125535408655),
 (1.7739794939756393, 1.7859295705954235),
 (1.7408700364828109, 1.7501551089021894),
 (1.7288502168655395, 1.7187697880797916),
 (1.689844034910202, 1.6910531851318147),
 (1.669118698835373, 1.6659734668334325),
 (1.6496987628936768, 1.6441864508390427),
 (1.6293941867351531, 1.6242887194951374),
 (1.6168639010190964, 1.6057418637805514),
 (1.6217393636703492, 1.589626677731673),
 (1.578944970369339, 1.5741784630219142),
 (1.5730746364593506, 1.5598156701193915),
 (1.5589076536893844, 1.5486967722906007),
 (1.5480487024784089, 1.5361990015374289),
 (1.5463894748687743, 1.5241086859504382),
 (1.5303162789344789, 1.5143978528843987),
 (1.534213799238205, 1.506165826651785),
 (1.5103626608848573, 1.4976868675152462),
 (1.5051646214723586, 1.4884281394879024),
 (1.4970894515514375,

In [14]:
expected.shape

(10000,)

In [15]:
expected=np.reshape(expected,(10000,1))

In [16]:
predicted.shape

(10000, 1)

In [17]:
predicted

array([[3],
       [0],
       [8],
       ..., 
       [2],
       [5],
       [7]])

In [18]:
num=len(expected)
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        #if y_pred[i].all() == labels[i].all():
        if np.array_equal(expected[i],predicted[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "Cifar 10 images")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 Cifar 10 images
correct:  5398 wrong:  4602 error rate:  46.02 %
got correctly  53.98 %
